## 一、一个简单的例子
函数 y = 2 * xT x  关于列向量 x 求导
若有 n 维向量 x=[x1,x2,x3,...,xn]T, 函数 f(x) 相对于 x 的梯度是一个包含 n 个偏导数的向量:

In [51]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [52]:
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
x.grad  # 默认值是None

In [53]:
y = 2 * torch.dot(x, x)
y  #  此时 y 是一个标量

tensor(28., grad_fn=<MulBackward0>)

In [54]:
# 调用反向传播函数来自动计算y关于x每个分量的梯度，并打印这些梯度。
y.backward()  # y = 2x^2  ->  y' = 4x
x.grad

tensor([ 0.,  4.,  8., 12.])

In [55]:
x.grad == 4 * x

tensor([True, True, True, True])

In [56]:
# 旧梯度需要被清除
x.grad.zero_()

y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

## 二、非标量变量的反向传播


In [57]:
x.grad.zero_()
y = x * x
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

## 三、分离计算

In [58]:
# 若有 y = y(x) ,z = z(y,x)
# 求 z 的梯度时，希望把 y 看作是一个常数，而非y(x),可用 .detach() 打断梯度的流向
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u  # (ux)' = u

tensor([True, True, True, True])

In [59]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## 四、Python控制流的梯度计算
 即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度。

In [76]:
# while 循环的迭代次数和 if 语句的结果都取决于输入 a 的值。
def f(a):
    b = a * 2
    while b.norm() < 1000: b *= 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [78]:
a = torch.randn(size=(),requires_grad=True)
d = f(a)
d.backward()

In [81]:
a,a.grad,a.grad == d / a

(tensor(-0.9693, requires_grad=True), tensor(204800.), tensor(True))